In [1]:
from sentence_transformers import SentenceTransformer, util
import os
# from utils import freq
import re
import pandas as pd
import numpy as np

d:\Program\Anaconda\envs\CP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [123]:
df_posts = pd.read_csv('all_posts.csv',  encoding='utf-8', dtype={'user_id':str})
# 把nan变为None
# df_posts = df_posts.where(pd.notnull(df_posts), None)

df_posts.shape

(15664, 19)

In [137]:
# 按照日期限制范围
def get_post_date_range(df,start,end,cluster = None):
    if cluster:
        df_cluster = df[df['cluster'] == cluster]
    else:
        df_cluster = df
    df_date = df_cluster[(df_cluster['publish_time'] >= start) & (df_cluster['publish_time'] <= end)]
    return df_date
# 按照cluster统计每组的数量
def get_post_gorup_count(df,group = 'cluster'):
    df_cluster_count = df.groupby(group).size().reset_index(name='count')
    return df_cluster_count
get_post_gorup_count(df_posts)
# 根据文本列表查找
def get_post_by_text_list(df,text_list):
    df_text = df[df['text'].str.contains('|'.join(text_list))]
    return df_text
# 把df_snccll的时间随机到'2021-04-20','2021-04-29'，注意要带上时分秒
def random_date(df,start,end):
    df['publish_time'] = pd.to_datetime(df['publish_time'], format='%Y-%m-%d %H:%M:%S')
    df['publish_time'] = df['publish_time'].apply(lambda x: x.replace(year=2021, month=4, day=np.random.randint(20, 30)))
    df['publish_time'] = df['publish_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    return df
def get_post_date_range(df,start,end,cluster = None):
    if cluster:
        df_cluster = df[df['cluster'] == cluster]
    else:
        df_cluster = df
    df_date = df_cluster[(df_cluster['publish_time'] >= start) & (df_cluster['publish_time'] <= end)]
    return df_date
# 按照日期限制范围
def get_post_date_out_range(df,start,end,cluster = None):
    if cluster:
        df_cluster = df[df['cluster'] == cluster]
    else:
        df_cluster = df
    df_date = df_cluster[(df_cluster['publish_time'] < start) | (df_cluster['publish_time'] > end)]
    return df_date

# japan_dead_fish

In [125]:
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(1, 19)

In [126]:
# 查找 2023.12.1 -> 2023.12.10之间，twitter上query为japan dead fish的
df_data_twitter = df_posts[df_posts['from'] == 'twitter']
df_data_twitter = get_post_date_range(df_data_twitter,'2023-12-01','2023-12-10')
df_data_twitter = df_data_twitter[df_data_twitter['query']== "Japan dead fish"]
df_data_twitter.shape
# 把这些data的cluster设置为japan_dead_fish
df_data_twitter['cluster'] = 'japan_dead_fish'
# 更新这些数据在df_posts
df_posts.update(df_data_twitter)
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(12, 19)

In [127]:
df_data_weibo = df_posts[df_posts['from'] == 'weibo']
df_data_weibo = get_post_date_range(df_data_weibo,'2023-12-01','2023-12-10')
df_data_weibo = df_data_weibo[df_data_weibo['query'] == "w9"]
df_data_weibo.shape
# 把这些data的cluster设置为japan_dead_fish
df_data_weibo['cluster'] = 'japan_dead_fish'
# 更新这些数据在df_posts
df_posts.update(df_data_weibo)
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(13, 19)

In [128]:
# 保存
df_posts.to_csv('all_posts.csv',index=False, encoding='utf-8')

In [143]:
# 读取all_posts.csv

df_posts = pd.read_csv('all_posts.csv',  encoding='utf-8', dtype={'user_id':str})
df = df_posts[df_posts['from'] =="twitter"]

In [156]:
get_post_date_range(df,'2021-04-20','2021-04-23','foreign_affairs_questions')

,from,post_id,text_trans,text,publish_time,cnt_retweet,cnt_comment,cnt_agree,user_id,screen_name,name,url,action,avatar,img,query,sentiment_score,cluster,event
14426,twitter,1384437636375347200,First of all， I think that it is not possible ...,まず自国のやっていることを注視したほうが良いかと2度も頭を丸められないと思いますが参考まで ...,2021-04-20 17:23:48,0.0,0.0,0.0,1204670046007545856,NaN,Ytse Jam@読了記事シェア,https://twitter.com/None/status/13844376363753...,post,NaN,NaN,background,0.0,foreign_affairs_questions,NaN
14429,twitter,1385223184736153600,The Chinese Ministry of Foreign Affairs remind...,中国外交部提醒日本人类只有一个地球 核废水处置不是日本私事,2021-04-22 21:25:18,1.0,4.0,1.0,1077993061135933440,GUTS_1997,Chikawa🇨🇳,https://twitter.com/GUTS_1997/status/138522318...,post,https://pbs.twimg.com/profile_images/173842273...,NaN,background,0.0,foreign_affairs_questions,NaN
14431,twitter,1384328900105306112,On the 19th， the Japanese government announced...,日本政府19日宣布因被检测出放射性物质超标福岛县海域的黑鲉被禁止上市 本月1日在福岛县海域捕...,2021-04-20 10:11:44,0.0,0.0,0.0,768384855927382016,NaN,hertz_1987,https://twitter.com/None/status/13843289001053...,post,NaN,NaN,background,0.0,foreign_affairs_questions,NaN
14434,twitter,1385178903388377088,A Korean professor has condemned Japan's unila...,A Korean professor has condemned Japan's unila...,2021-04-22 18:29:20,16.0,33.0,3.0,487118986,XHNews,China Xinhua News,https://twitter.com/XHNews/status/138517890338...,post,https://pbs.twimg.com/profile_images/124668601...,NaN,background,0.0,foreign_affairs_questions,NaN
14437,twitter,1384451824229380096,RT processing: China net no longer goes to Jap...,rt 処理水海洋放出の決定に中国ネットもう日本には行かない,2021-04-20 18:20:11,1961.0,0.0,0.0,913763875262042112,NaN,やごやご,https://twitter.com/None/status/13844518242293...,post,NaN,NaN,background,0.0,foreign_affairs_questions,NaN
14438,twitter,1385110255684644864,On April 21， 2021， during a video consultation...,2021年4月21日国务委员兼外长王毅同德国外长马斯举行视频磋商时提及日本福岛核污染废水排放...,2021-04-22 13:56:33,0.0,0.0,1.0,1248589013604167681,beroshima309,鳴門支那竹,https://twitter.com/beroshima309/status/138511...,post,https://pbs.twimg.com/profile_images/124859495...,NaN,background,0.0,foreign_affairs_questions,NaN
14444,twitter,1390041819048677376,idiots world japannuclearwater,idiots world japannuclearwater,2021-04-21 18:18:35,9.0,21.0,3.0,820114986,zzxang86,Joey ZHANG,https://twitter.com/zzxang86/status/1390041819...,post,https://pbs.twimg.com/profile_images/260319188...,NaN,background,0.0,foreign_affairs_questions,NaN
14447,twitter,1384387338654994432,Antony Blinken is also a real dog. The investi...,布林肯也是真的狗 不开放美国实验室泄露病毒的调查 不处置日本核废水 偏偏甩锅中国搞什么狗屁气候问题,2021-04-20 14:03:56,0.0,1.0,0.0,1377291348483579906,hit_wangye,sh,https://twitter.com/hit_wangye/status/13843873...,post,https://pbs.twimg.com/profile_images/137729157...,NaN,background,0.0,foreign_affairs_questions,NaN
14448,twitter,1384478345312624640,RT processing: China net no longer goes to Jap...,rt 処理水海洋放出の決定に中国ネットもう日本には行かない,2021-04-20 20:05:34,1961.0,0.0,0.0,110676340,NaN,♨️マスクワクチン・コオロギ禍🎉,https://twitter.com/None/status/13844783453126...,post,NaN,NaN,background,0.0,foreign_affairs_questions,NaN
14449,twitter,1384927754114699264,The decision of the Kan administration should ...,rt これで汚染水を海洋放出する気なのか菅政権の判断は間違っている風評被害どころの話じゃない...,2021-04-22 01:51:21,18.0,0.0,0.0,1038293163616567296,NaN,2488usagi,https://twitter.com/None/status/13849277541146...,post,NaN,NaN,background,0.0,foreign_affairs_questions,NaN
